## 1.	Is it OK to initialize all the weights to the same value as long as that value is selected randomly using He initialization?

No, all weights should be sampled independently; they should not
all have the same initial value. One important goal of sampling
weights randomly is to break symmetry: if all the weights have
the same initial value, even if that value is not zero, then
symmetry is not broken (i.e., all neurons in a given layer are
equivalent), and backpropagation will be unable to break it.
Concretely, this means that all the neurons in any given layer will
always have the same weights. It’s like having just one neuron per
layer, and much slower. It is virtually impossible for such a
configuration to converge to a good solution.

## 2.	Is it OK to initialize the bias terms to 0?

No, all weights should be sampled independently; they should not
all have the same initial value. One important goal of sampling
weights randomly is to break symmetry: if all the weights have
the same initial value, even if that value is not zero, then
symmetry is not broken (i.e., all neurons in a given layer are
equivalent), and backpropagation will be unable to break it.
Concretely, this means that all the neurons in any given layer will
always have the same weights. It’s like having just one neuron per
layer, and much slower. It is virtually impossible for such a
configuration to converge to a good solution.

## 3.	Name three advantages of the SELU activation function over ReLU.

A few advantages of the SELU function over the ReLU function
are:

It can take on negative values, so the average output of
the neurons in any given layer is typically closer to zero
than when using the ReLU activation function (which
never outputs negative values). This helps alleviate the
vanishing gradients problem.

It always has a nonzero derivative, which avoids the
dying units issue that can affect ReLU units.

When the conditions are right (i.e., if the model is
sequential, and the weights are initialized using LeCun
initialization, and the inputs are standardized, and there’s
no incompatible layer or regularization, such as dropout
or ℓ regularization), then the SELU activation function
ensures the model is self-normalized, which solves the
exploding/vanishing gradients problems.

## 4.	In which cases would you want to use each of the following activation functions: SELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?

The SELU activation function is a good default. If you need the
neural network to be as fast as possible, you can use one of the
leaky ReLU variants instead (e.g., a simple leaky ReLU using the
default hyperparameter value). The simplicity of the ReLU
activation function makes it many people’s preferred option,
despite the fact that it is generally outperformed by SELU and
leaky ReLU. However, the ReLU activation function’s ability to
output precisely zero can be useful in some cases. Moreover, it can sometimes benefit from optimized
implementation as well as from hardware acceleration. The
hyperbolic tangent (tanh) can be useful in the output layer if you
need to output a number between –1 and 1, but nowadays it is not
used much in hidden layers (except in recurrent nets). The logistic
activation function is also useful in the output layer when you
need to estimate a probability (e.g., for binary classification), but
is rarely used in hidden layers (there are exceptions—for
example, for the coding layer of variational autoencoders. Finally, the softmax activation function is useful in
the output layer to output probabilities for mutually exclusive
classes, but it is rarely (if ever) used in hidden layers.

## 5.	What may happen if you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer?

If you set the momentum hyperparameter too close to 1 (e.g.,
0.99999) when using an SGD optimizer, then the algorithm will
likely pick up a lot of speed, hopefully moving roughly toward the
global minimum, but its momentum will carry it right past the
minimum. Then it will slow down and come back, accelerate
again, overshoot again, and so on. It may oscillate this way many
times before converging, so overall it will take much longer to
converge than with a smaller momentum value.

## 6.	Name three ways you can produce a sparse model.

One way to produce a sparse model (i.e., with most weights equal
to zero) is to train the model normally, then zero out tiny weights.
For more sparsity, you can apply ℓ regularization during training,
which pushes the optimizer toward sparsity. A third option is to
use the TensorFlow Model Optimization Toolkit.

## 7.	Does dropout slow down training? Does it slow down inference (i.e., making predictions on new instances)? What about MC Dropout?

Yes, dropout does slow down training, in general roughly by a
factor of two. However, it has no impact on inference speed since
it is only turned on during training. MC Dropout is exactly like
dropout during training, but it is still active during inference, so
each inference is slowed down slightly. More importantly, when
using MC Dropout you generally want to run inference 10 times
or more to get better predictions. This means that making
predictions is slowed down by a factor of 10 or more.


## 8.	Practice training a deep neural network on the CIFAR10 image dataset:

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## a: Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.

In [2]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

## b. Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_ data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you’ll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model’s architecture or hyperparameters.

In [3]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [4]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [6]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [7]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

Reusing TensorBoard on port 6006 (pid 9544), started 1:03:17 ago. (Use '!kill 9544' to kill it.)

In [8]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 49s 22ms/step - loss: 4.0822 - accuracy: 0.1705 - val_loss: 2.1179 - val_accuracy: 0.2326
Epoch 2/100
1407/1407 [==============================] - 29s 21ms/step - loss: 2.0592 - accuracy: 0.2462 - val_loss: 2.1213 - val_accuracy: 0.2340
Epoch 3/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.9363 - accuracy: 0.2911 - val_loss: 2.0175 - val_accuracy: 0.2702
Epoch 4/100
1407/1407 [==============================] - 31s 22ms/step - loss: 1.8588 - accuracy: 0.3209 - val_loss: 1.8617 - val_accuracy: 0.3344
Epoch 5/100
1407/1407 [==============================] - 33s 23ms/step - loss: 1.7975 - accuracy: 0.3451 - val_loss: 1.8554 - val_accuracy: 0.3308
Epoch 6/100
1407/1407 [==============================] - 33s 24ms/step - loss: 1.7525 - accuracy: 0.3660 - val_loss: 1.7481 - val_accuracy: 0.3714
Epoch 7/100
1407/1407 [==============================] - 32s 22ms/step - loss: 1.7116 - accuracy: 0.3770 - val_loss: 1

In [9]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 1s 4ms/step - loss: 1.5223 - accuracy: 0.4720


[1.522278070449829, 0.47200000286102295]

## c.	Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

In [10]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 59s 31ms/step - loss: 1.8452 - accuracy: 0.3377 - val_loss: 1.6777 - val_accuracy: 0.4086
Epoch 2/100
1407/1407 [==============================] - 43s 31ms/step - loss: 1.6688 - accuracy: 0.4072 - val_loss: 1.6004 - val_accuracy: 0.4238
Epoch 3/100
1407/1407 [==============================] - 42s 30ms/step - loss: 1.5976 - accuracy: 0.4323 - val_loss: 1.5543 - val_accuracy: 0.4350
Epoch 4/100
1407/1407 [==============================] - 43s 30ms/step - loss: 1.5480 - accuracy: 0.4474 - val_loss: 1.5040 - val_accuracy: 0.4664
Epoch 5/100
1407/1407 [==============================] - 43s 30ms/step - loss: 1.4991 - accuracy: 0.4666 - val_loss: 1.4577 - val_accuracy: 0.4784
Epoch 6/100
1407/1407 [==============================] - 44s 31ms/step - loss: 1.4638 - accuracy: 0.4787 - val_loss: 1.4232 - val_accuracy: 0.4912
Epoch 7/100
1407/1407 [==============================] - 42s 30ms/step - loss: 1.4301 - accuracy: 0.4899 - val_loss: 1

[1.315259575843811, 0.5378000140190125]

## d.	Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

In [ ]:
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

## e.	Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

In [ ]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [ ]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [ ]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy